# <center>String Output Parser

The `StrOutputParser` is the simplest output parser in LangChain. It is used to parse the output of a Language Model (LLM) and return it as a plain string.

- preffered to be used when we want to convo with out model more than 1 time

#### example workflow
- topic -> LLM -> [detail information] -> LLM -> [5 line summarize]

## <center>doing without Str Output Parser

In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

In [2]:
load_dotenv()

True

In [3]:
repo_id="google/gemma-2-2b-it"

In [4]:
llm = HuggingFaceEndpoint(
    repo_id = repo_id,
    task="text-generation",
    verbose=True,
    huggingfacehub_api_token=os.environ['HUGGINGFACE_ACCESS_TOKEN']
)

In [5]:
model = ChatHuggingFace(llm=llm)

In [6]:
# 1st prompt -> detail report
template1 = PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"]
)


In [7]:
# 2nd Prompt -> summary
template2 = PromptTemplate(
    template="Write a 5 line summary on the following text. /n{text}",
    input_variables = ['text']
    
)

In [8]:
# prompt1 = template1.invoke({"topic": "black hole"})
# or
prompt1 = template1.format(topic= "black hole")

In [9]:
prompt1

'Write a detailed report on black hole'

In [10]:
result = model.invoke(prompt1)

In [11]:
result

AIMessage(content="## A Deep Dive into Black Holes\n\nBlack holes, the enigmatic remnants of massive stars that have collapsed under their own gravity, are theorized objects of extreme density and warping spacetime. These cosmic beasts defy easy description and offer captivating insights into the fundamental laws of physics. \n\nThis report delves into the fascinating world of black holes, exploring their formation, properties, observation, and the ongoing scientific research surrounding them. \n\n**1. Formation:**\n\n-  **Death of a Star:** Black holes arise from the gravitational implosion of massive stars (at least 3 times the mass of our sun) at the end of their life cycle. A supernova event occurs, but due to immense gravity, the core implodes, crushing its massive mass into an infinitely small, dense point called a singularity.\n- **Escape Velocity:** This event results in immense gravitational potential, surpassing the escape velocity of even light.  Any matter that falls toward

In [12]:
prompt2 = template2.format(text=result.content)

In [13]:
result = model.invoke(prompt2)

In [14]:
result.text

<bound method BaseMessage.text of AIMessage(content='Black holes are hyperdense remnants of massive stars that possess immense gravity, creating an event horizon beyond which nothing can escape.  They form from the collapse of stellar cores, and their properties, including mass, spin, and the event horizon (Schwarzschild radius), are studied using techniques like gravitational lensing, accretion disks, and X-ray/Gamma-ray emissions. With the detection of gravitational waves and the contributions of scientists investigating Hawking radiation and singularities, understanding black holes continues to be a key point of research, shedding light on the fabric of spacetime and the evolution of the universe.  As we delve deeper into these enigmatic objects, we gain invaluable insights into the physical laws and ontological mysteries that remain a captivating field of study. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 949, 'total_toke

## <center>doing with Str Output Parser¶

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [16]:
load_dotenv()

True

In [17]:
API_KEY = os.environ['GEMINI_API_KEY']
# MODEL_NAME = "gemini-1.5-flash"
MODEL_NAME = "gemini-2.0-flash"

In [18]:
model = ChatGoogleGenerativeAI(
    api_key=API_KEY,
    verbose=True,
    model=MODEL_NAME,
)

In [19]:
# 1st prompt -> detail report
template1 = PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"]
)

In [20]:
# 2nd Prompt -> summary
template2 = PromptTemplate(
    template="Write a 5 line summary on the following text. /n{text}",
    input_variables = ['text']   
)

In [21]:
parser = StrOutputParser()

In [22]:
chain = template1 | model | parser | template2 | model | parser

In [23]:
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Write a detailed report on {topic}')
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022610672530>, default_metadata=(), model_kwargs={})
| StrOutputParser()
| PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write a 5 line summary on the following text. /n{text}')
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022610672530>, default_metadata=(), model_kwargs={})
| StrOutputParser()

In [24]:
chain.invoke({"topic": "black hole"})

'Black holes are regions of spacetime with extreme gravity, preventing even light from escaping. They form primarily from stellar collapse or direct collapse, resulting in stellar, intermediate, or supermassive black holes. Key properties include the event horizon, singularity, mass, charge, and spin.  Detection relies on indirect methods like gravitational lensing, X-ray emission, gravitational waves, and stellar orbits. Ongoing research focuses on SMBH formation, the information paradox, and testing general relativity.'